In [1]:
import json
import time

import gurobipy as gp
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline
import numpy as np
import pandas as pd
from gurobipy import GRB
from tqdm import tqdm

from orc.branch import (
    branch_beasley, branch_reduced_costs, 
    branch_lp
)
from orc.callbacks import (
    ColumnInclusionCallback,
    LagrPenaltiesReductionCallback,
    PrimalHeurCallback
)
from orc.data_structures import (
    BranchAndBound, TimeLimitException
)
from orc.primal import dobson, greedy, hall_hochbaum
from orc.relaxation import subgrad_opt, lp_rel
from orc.utils import generate_problem

mpl.rcParams["font.family"] = ["serif"]
mpl.rcParams["font.serif"] = ["cmr10"]
matplotlib_inline.backend_inline.set_matplotlib_formats("retina")

# Primal heuristics comparisons

## Without fixed variables

In [ ]:
table = []
funcs = [(greedy, "Greedy"), (dobson, "Dobson"),
         (hall_hochbaum, "Hall-Hochbaum")]
for rows, cols in tqdm(
    [(5, 10), (10, 20), (20, 50), (50, 100)], leave=False):
    res = {"Rows": rows, "Cols": cols, "Greedy": 0, 
           "Dobson": 0, "Hall-Hochbaum": 0}
    for i in tqdm(range(10)):
        A, b = generate_problem(rows, cols)
        v = []
        for f, name in funcs:
            ub = np.sum(A, axis=0) @ f(A, b, [], [])
            v.append(ub)
        j = np.argmin(v)
        best = funcs[j][1]
        res[best] += 1
    table.append(res)

In [ ]:
with open("results/primal.json", "w") as f:
    out = {"output": table}
    json.dump(out, f)

In [ ]:
with open("results/primal.json", "r") as f:
    table = json.load(f)["output"]

In [ ]:
df = pd.DataFrame(table)
df = df.set_index(["Rows", "Cols"])
df

In [ ]:
df.style.to_latex(column_format="llccc")

## With fixed variables

In [ ]:
np.random.seed(42)

table = []
funcs = [(greedy, "Greedy"), (dobson, "Dobson"),
         (hall_hochbaum, "Hall-Hochbaum")]
for rows, cols in tqdm(
    [(5, 10), (10, 20), (20, 50), (50, 100)], leave=False):
    res = {"Rows": rows, "Cols": cols, "Greedy": 0, 
           "Dobson": 0, "Hall-Hochbaum": 0}
    for i in tqdm(range(10)):
        A, b = generate_problem(rows, cols)
        fixed_n = int(cols * 0.3) 
        fixed = np.random.choice(
            A.shape[-1], fixed_n, replace=False)
        x0 = fixed[:int(fixed_n / 2)]
        x1 = fixed[int(fixed_n / 2) + 1:]
        x = np.ones(A.shape[-1])
        x[x0] = 0
        if np.any(A @ x < b):
            continue
        v = []
        for f, name in funcs:
            ub = np.sum(A, axis=0) @ f(A, b, [], [])
            v.append(ub)
        j = np.argmin(v)
        best = funcs[j][1]
        res[best] += 1
    table.append(res)

In [ ]:
table

# Subgradient optimization lower bounds

In [ ]:
data = {}
for rows, cols in tqdm(
    [(10, 20), (20, 50), (40, 80), (80, 150)], leave=False):
    A, b = generate_problem(rows, cols)
    x = hall_hochbaum(A, b, [], [])
    ub = np.sum(A, axis=0) @ x

    res = []
    for omega in [10, 20, 40, 50, 70, 100, 200, 500]:
        lb = subgrad_opt(A, b, ub, [], [], omega=omega)
        res.append([omega, lb])
    data[(rows, cols)] = res

In [ ]:
fig, ax = plt.subplots()
for k, v in data.items():
    x = np.array(v)[:,0]
    y = np.array(v)[:,1]
    y = (y - y.min()) / (y.max() - y.min())
    ax.plot(x, y, label=f"{k[0]} rows, {k[1]} cols")
ax.legend()
ax.set_xlabel("Iterations")
ax.set_ylabel("Scaled lower bound")
ax.set_title("Subgradient optimization lower bounds")
plt.savefig("report/img/subgrad_lb.png", 
            dpi=300, bbox_inches="tight")

# Branch and Bound models

In [2]:
lagr_callback = LagrPenaltiesReductionCallback()
col_callback = ColumnInclusionCallback()
primal_heur = PrimalHeurCallback()
primal_heur_root = PrimalHeurCallback(only_root=True)

models = {
    "Subgrad": {
        "branch_strategy": branch_reduced_costs, 
        "lb_strategy": subgrad_opt
        },
    "SubgradPrimal": {
        "branch_strategy": branch_reduced_costs, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur]
        },
    "SubgradPrimalRed": {
        "branch_strategy": branch_reduced_costs, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur, lagr_callback, col_callback]
        },
    "SubgradPrimalRootRed": {
        "branch_strategy": branch_reduced_costs, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur_root, lagr_callback, col_callback]
        },
    "SubgradPrimalRedBeasleyBranch": {
        "branch_strategy": branch_beasley, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur, lagr_callback, col_callback]
        },
    "LPPrimalRed": {
        "branch_strategy": branch_lp, 
        "lb_strategy": lp_rel,
        "callbacks": [primal_heur, col_callback]}
}

In [3]:
data = {}
time_limit = 60 * 5
for rows, cols, density in tqdm(
    [(5, 10, 0.3), (5, 10, 0.5), (5, 10, 0.7), 
     (10, 20, 0.3), (10, 20, 0.5), (10, 20, 0.7),
     (20, 50, 0.3), (20, 50, 0.5), (20, 50, 0.7),
     (50, 100, 0.3), (50, 100, 0.5), (50, 100, 0.7)],
     desc="Problem:", leave=False):
    res = {}
    A, b = generate_problem(rows, cols, density)
    
    start = time.process_time()
    m = gp.Model()
    m.Params.LogToConsole = 0
    x = m.addMVar(A.shape[-1], vtype=GRB.BINARY, name="x")
    m.setObjective(np.sum(A, axis=0) @ x)
    m.addConstr(A @ x >= b)
    m.optimize()
    elapsed = time.process_time() - start
    opt_gurobi = m.getObjective().getValue()
    
    x = []
    for v in m.getVars():
        x.append(v.x)
    x = np.array(x)
    
    # Make sure that the solution is feasible
    assert np.all(A @ x >= b)

    res["Gurobi"] = (elapsed, m.NodeCount)

    for model, config in tqdm(
        models.items(), desc="Model:", leave=False):
        start = time.process_time()
        bb = BranchAndBound(**config, time_limit=time_limit)
        try:
            bb.search(A, b)
            elapsed = time.process_time() - start
            x = np.zeros(A.shape[-1])
            x[bb.best.x1] = 1
            
            # Make sure that the solution is feasible
            assert np.all(A @ x >= b)
            opt = np.sum(A, axis=0) @ x

            # Check that the solution is optimal
            assert opt == opt_gurobi
            res[model] = (elapsed, bb.node_count)
        except TimeLimitException:
            res[model] = (time_limit, bb.node_count)
        

    data[(rows, cols, density)] = res

Problem::   0%|          | 0/12 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-16


Problem::  50%|█████     | 6/12 [17:51<29:06, 291.14s/it]

In [ ]:
with open("results.json", "w") as f:
    nd = {str(k): v for k, v in data.items()}
    json.dump(nd, f)

In [ ]:
with open("results.json", "r") as f:
    results = json.load(f)

In [ ]:
time_dict = {
    tuple(k[1:-1].split(", ")): {
        model: values[0] for model, values in v.items()}  
    for k, v in results.items()}
df = pd.DataFrame(time_dict)
df = df.T
df = df.rename_axis(["Rows", "Cols", "Density"])
df

In [ ]:
df.style.highlight_min(axis=0, props="font-weight:bold;").to_latex(convert_css=True)

In [ ]:
nodes_dict = {
    tuple(k[1:-1].split(", ")): {
        model: values[1] for model, values in v.items()}  
    for k, v in results.items()}
df = pd.DataFrame(nodes_dict)
df = df.T
df = df.rename_axis(["Rows", "Cols", "Density"])
df